In [50]:
import pandas as pd
import numpy as np

In [78]:
base_path = "D:\\summer 2_nd\\Formula 1\\"
csv_prefix = [
    "circuits", "constructors", "constructor_results", "constructor_standings",
    "drivers", "driver_standings", "lap_times", "pit_stops", "qualifying",
    "races", "results", "seasons", "sprint_results", "status"
]

for name in csv_prefix:
    globals()[name] = pd.read_csv(f"{base_path}{name}.csv")


In [80]:
drivers.columns = ["driverId","driverRef","driver_number","driver_code","driver_forename","driver_surname","driver_dob","driver_nationality","driver_url"]
driver_standings.columns = ["driverStandingsId","raceId","driverId","driverst_points","driverst_position","driverst_positionText","driverst_wins"]
lap_times.columns = ["raceId","driverId","lap", "lap_position", "lap_time","lap_milliseconds"]

constructors.columns = ['constructorId', 'constructorRef', 'cs_name', 'cs_nationality', 'cs_url']
constructor_results.columns = ['constructorResultsId', 'raceId', 'constructorId', 'csr_points', 'csr_status']
constructor_standings.columns =['constructorStandingsId', 'raceId', 'constructorId', 'css_points',
       'css_position', 'css_positionText', 'css_wins']

races.columns = ['raceId', 'year', 'round', 'circuitId', 'race_name', 'race_date', 'race_time', 'race_url',
       'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
       'quali_date', 'quali_time', 'sprint_date', 'sprint_time']
pit_stops.columns = ['raceId', 'driverId', 'pit_stop', 'pit_lap', 'pit_time', 'pit_duration', 'pit_milliseconds']
qualifying.columns =['qualifyId', 'raceId', 'driverId', 'constructorId', 'qs_number',
       'position', 'q1', 'q2', 'q3']
results.columns =['resultId', 'raceId', 'driverId', 'constructorId', 'result_number', 'result_grid',
       'result_position', 'result_positionText', 'result_positionOrder', 'result_points', 'result_laps', 'result_time',
       'result_milliseconds', 'result_fastestLap', 'result_rank', 'result_fastestLapTime',
       'result_fastestLapSpeed', 'statusId']

In [4]:
pit_stops['pit_duration'] = pd.to_numeric(pit_stops['pit_duration'], errors='coerce')

lap_summary = lap_times.groupby(['raceId', 'driverId']).agg({
    'lap': 'max',  
    'lap_milliseconds': 'mean'  
}).rename(columns={
    'lap': 'total_laps',
    'lap_milliseconds': 'avg_lap_time'
}).reset_index()

pit_summary = pit_stops.groupby(['raceId', 'driverId']).agg({
    'pit_stop': 'count',         
    'pit_duration': 'mean'     
}).rename(columns={
    'pit_stop': 'num_pit_stops',
    'pit_duration': 'avg_pit_duration'
}).reset_index()


In [83]:
all_drivers["driver_surname"].unique()

array(['Hamilton', 'Heidfeld', 'Rosberg', 'Alonso', 'Kovalainen',
       'Nakajima', 'Bourdais', 'Räikkönen', 'Kubica', 'Glock', 'Sato',
       'Piquet Jr.', 'Massa', 'Coulthard', 'Trulli', 'Sutil', 'Webber',
       'Button', 'Davidson', 'Vettel', 'Fisichella', 'Barrichello',
       'Schumacher', 'Liuzzi', 'Wurz', 'Speed', 'Albers', 'Yamamoto',
       'Pablo Montoya', 'Klien', 'Monteiro', 'Ide', 'Villeneuve',
       'Montagny', 'de la Rosa', 'Doornbos', 'Karthikeyan', 'Friesacher',
       'Zonta', 'Pizzonia', 'da Matta', 'Panis', 'Pantano', 'Bruni',
       'Baumgartner', 'Gené', 'Frentzen', 'Verstappen', 'Wilson',
       'Firman', 'Kiesa', 'Burti', 'Alesi', 'Irvine', 'Häkkinen',
       'Marques', 'Bernoldi', 'Mazzacane', 'Enge', 'Yoong', 'Salo',
       'Diniz', 'Herbert', 'McNish', 'Buemi', 'Takagi', 'Badoer',
       'Zanardi', 'Hill', 'Rosset', 'Tuero', 'Nakano', 'Magnussen',
       'Berger', 'Larini', 'Katayama', 'Morbidelli', 'Fontana', 'Lamy',
       'Brundle', 'Montermini', 'Lavag

In [84]:

all_drivers = None
all_constructors = None
all_races = None

all_drivers = pd.merge(drivers, driver_standings, on="driverId", how="inner")


all_constructors = pd.merge(constructor_results, constructor_standings,
                            on=["constructorId", "raceId"], how="inner")
all_constructors = pd.merge(all_constructors, constructors,
                            on="constructorId", how="inner")

In [85]:
all_races = pd.merge(races,all_drivers,on="raceId", how="inner")
all_races = pd.merge(all_races,all_constructors,on="raceId", how="inner")
all_races = pd.merge(all_races,results,on=["raceId","driverId","constructorId"], how="inner")
all_races = pd.merge(all_races,qualifying,on=["raceId","driverId","constructorId"], how="inner")
all_races = pd.merge(all_races, lap_summary, on=["raceId", "driverId"], how="left")
all_races = pd.merge(all_races, pit_summary, on=["raceId", "driverId"], how="left")
required_races = all_races.copy()
required_races = required_races[required_races.year.isin([2021,2022, 2023, 2024])]
required_races.drop(columns=["driver_url", "cs_url","race_url"], inplace=True)
required_races.reset_index(drop=True, inplace=True)


In [95]:
required_races.loc[required_races["driverRef"] == "max_verstappen"]


,raceId,year,round,circuitId,race_name,race_date,race_time,driverId,driverRef,driver_dob,driver_nationality,driverst_points,driverst_position,driverst_wins,constructorId,csr_points,csr_status,constructorStandingsId,css_points,css_position,css_wins,constructorRef,cs_nationality,result_number,result_grid,result_position,result_positionText,result_positionOrder,result_points,result_laps,result_time,result_milliseconds,result_fastestLap,result_rank,result_fastestLapTime,result_fastestLapSpeed,qs_number,position,q1,q2,q3,total_laps,avg_lap_time,num_pit_stops,avg_pit_duration
8,1053,2021,2,21,Emilia Romagna Grand Prix,2021-04-18,13:00:00,830,max_verstappen,1997-09-30,Dutch,43.0,2,1,9,25.0,\N,27704,53.0,2,1,red_bull,Austrian,33,3,1,1,1,25.0,63,2:02:34.598,7354598,60,2,1:17.524,227.960,33,3,1:15.109,1:14.884,1:14.498,63.0,116739.650794,2.0,29.809000
28,1074,2022,1,3,Bahrain Grand Prix,2022-03-20,15:00:00,830,max_verstappen,1997-09-30,Dutch,0.0,19,0,9,0.0,\N,28142,0.0,10,0,red_bull,Austrian,1,2,19,19,19,0.0,54,\N,\N,51,2,1:35.440,204.140,1,2,1:31.785,1:30.757,1:30.681,54.0,103661.037037,3.0,24.745333
48,1052,2021,1,3,Bahrain Grand Prix,2021-03-28,15:00:00,830,max_verstappen,1997-09-30,Dutch,18.0,2,0,9,28.0,\N,27674,28.0,2,0,red_bull,Austrian,33,1,2,2,2,18.0,56,+0.745,5524642,41,2,1:33.228,208.984,33,1,1:30.499,1:30.318,1:28.997,56.0,98654.321429,2.0,24.307500
68,1051,2021,20,78,Qatar Grand Prix,2021-11-21,14:00:00,830,max_verstappen,1997-09-30,Dutch,351.5,1,9,9,31.0,\N,27904,541.5,2,10,red_bull,Austrian,33,7,2,2,2,19.0,57,+25.743,5094214,57,1,1:23.196,232.799,33,2,1:21.996,1:21.984,1:21.282,57.0,89372.175439,3.0,25.313333
88,1054,2021,3,75,Portuguese Grand Prix,2021-05-02,14:00:00,830,max_verstappen,1997-09-30,Dutch,61.0,2,1,9,30.0,\N,27714,83.0,2,1,red_bull,Austrian,33,3,2,2,2,18.0,66,+29.148,5700569,62,3,1:20.695,207.581,33,3,1:19.485,1:18.650,1:18.746,66.0,86372.257576,4.0,24.471000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,1140,2024,20,32,Mexico City Grand Prix,2024-10-27,20:00:00,830,max_verstappen,1997-09-30,Dutch,362.0,1,7,9,8.0,\N,28933,512.0,3,7,red_bull,Austrian,1,2,6,6,6,8.0,71,+59.558,6115358,53,13,1:20.872,191.591,1,2,1:16.803,1:16.514,1:16.171,71.0,86131.802817,1.0,43.730000
1724,1141,2024,21,18,São Paulo Grand Prix,2024-11-03,17:00:00,830,max_verstappen,1997-09-30,Dutch,393.0,1,8,9,32.0,\N,28943,544.0,3,8,red_bull,Austrian,1,17,1,1,1,26.0,69,2:06:54.430,7614430,67,1,1:20.472,192.767,1,12,1:28.522,1:27.771,\N,69.0,110354.057971,1.0,NaN
1745,1142,2024,22,80,Las Vegas Grand Prix,2024-11-23,06:00:00,830,max_verstappen,1997-09-30,Dutch,403.0,1,8,9,11.0,\N,28953,555.0,3,8,red_bull,Austrian,1,5,5,5,5,10.0,50,+16.582,4942551,43,6,1:36.248,231.938,1,5,1:33.299,1:33.085,1:32.797,50.0,98851.020000,2.0,21.475500
1765,1143,2024,23,78,Qatar Grand Prix,2024-12-01,17:00:00,830,max_verstappen,1997-09-30,Dutch,429.0,1,9,9,26.0,\N,28963,581.0,3,9,red_bull,Austrian,1,2,1,1,1,25.0,57,1:31:05.323,5465323,55,2,1:22.905,235.310,1,1,1:21.579,1:20.687,1:20.520,57.0,95882.859649,3.0,24.963333


In [6]:
pd.set_option('display.max_columns', None)
required_races.head()

NameError: name 'required_races' is not defined

In [93]:
required_races.drop(columns=["fp1_date","fp1_time","fp2_date","fp2_time","fp3_date","fp3_time",
                            "quali_date","quali_time","sprint_date","sprint_time","driver_number","driver_code",
   "driver_forename", "driver_surname", "driverStandingsId", "constructorResultsId","driverst_positionText" ,"css_positionText",
   "cs_name", "resultId", "statusId", "qualifyId"], inplace=True)

In [94]:
required_races  = required_races[required_races["result_position"].str.strip().str.isnumeric()]
required_races["result_position"] = required_races["result_position"].astype(int)

C:\Users\ibrah\AppData\Local\Temp\ipykernel_16728\593802534.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required_races["result_position"] = required_races["result_position"].astype(int)


In [10]:
required_races["driver_nationality"].unique()

array(['British', 'Spanish', 'Finnish', 'German', 'French', 'Mexican',
       'Australian', 'Dutch', 'Canadian', 'Italian', 'Monegasque',
       'Japanese', 'Russian', 'Danish', 'Thai', 'Chinese', 'Polish',
       'American', 'New Zealander', 'Argentinian '], dtype=object)

In [11]:
required_races.to_csv(f"{base_path}data_set.csv", index=False)

In [12]:
required_races = required_races[required_races["result_position"]!="\\N"]


In [96]:
required_races['driver_circuit_avg_result_point_before_race'] = (
    required_races
    .groupby(['driverId', 'circuitId'])['result_position']
    .transform(lambda x: x.shift(1).expanding().mean())
)
required_races['driver_circuit_avg_pitstops_before_race'] = (
    required_races
    .groupby(['driverId', 'circuitId'])['num_pit_stops']
    .transform(lambda x: x.shift(1).expanding().mean())
)



C:\Users\ibrah\AppData\Local\Temp\ipykernel_16728\4148442609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required_races['driver_circuit_avg_result_point_before_race'] = (
C:\Users\ibrah\AppData\Local\Temp\ipykernel_16728\4148442609.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required_races['driver_circuit_avg_pitstops_before_race'] = (


In [97]:
required_races["result_position"].info()

<class 'pandas.core.series.Series'>
Index: 1580 entries, 0 to 1797
Series name: result_position
Non-Null Count  Dtype
--------------  -----
1580 non-null   int32
dtypes: int32(1)
memory usage: 18.5 KB


In [98]:
required_races['driver_avg_finish_pos_prior'] = (
    required_races
    .groupby('driverId')['result_position']
    .transform(lambda x: x.shift(1).expanding().mean())
)
required_races['is_podium'] = (required_races['result_position'] <= 3).astype(int)

required_races['driver_podium_rate_prior'] = (
    required_races
    .groupby('driverId')['is_podium']
    .transform(lambda x: x.shift(1).expanding().mean())
)

required_races['is_win'] = (required_races['result_position'] == 1).astype(int)

required_races['driver_win_rate_prior'] = (
    required_races
    .groupby('driverId')['is_win']
    .transform(lambda x: x.shift(1).expanding().mean())
)
required_races['driver_avg_finish_last_3'] = (
    required_races.groupby('driverId')['result_position']
    .transform(lambda x: x.shift(1).rolling(window=3, min_periods=1).mean())
)

required_races = required_races.sort_values(by=['driverId', 'race_date'])

required_races['driver_experience_races'] = (
    required_races.groupby('driverId').cumcount()
)

required_races['driver_experience_at_circuit'] = (
    required_races
    .groupby(['driverId', 'circuitId'])
    .cumcount()
)


C:\Users\ibrah\AppData\Local\Temp\ipykernel_16728\802029032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required_races['driver_avg_finish_pos_prior'] = (
C:\Users\ibrah\AppData\Local\Temp\ipykernel_16728\802029032.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required_races['is_podium'] = (required_races['result_position'] <= 3).astype(int)
C:\Users\ibrah\AppData\Local\Temp\ipykernel_16728\802029032.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [ ]:
required_races["driver_nationality"].unique()

In [99]:
required_races['constructor_avg_finish_prior'] = (
    required_races.groupby('constructorId')['result_position']
    .transform(lambda x: x.shift(1).expanding().mean())
)



In [100]:
required_races.loc[required_races["driverRef"] == "max_verstappen"]


,raceId,year,round,circuitId,race_name,race_date,race_time,driverId,driverRef,driver_dob,driver_nationality,driverst_points,driverst_position,driverst_wins,constructorId,csr_points,csr_status,constructorStandingsId,css_points,css_position,css_wins,constructorRef,cs_nationality,result_number,result_grid,result_position,result_positionText,result_positionOrder,result_points,result_laps,result_time,result_milliseconds,result_fastestLap,result_rank,result_fastestLapTime,result_fastestLapSpeed,qs_number,position,q1,q2,q3,total_laps,avg_lap_time,num_pit_stops,avg_pit_duration,driver_circuit_avg_result_point_before_race,driver_circuit_avg_pitstops_before_race,driver_avg_finish_pos_prior,is_podium,driver_podium_rate_prior,is_win,driver_win_rate_prior,driver_avg_finish_last_3,driver_experience_races,driver_experience_at_circuit,constructor_avg_finish_prior
48,1052,2021,1,3,Bahrain Grand Prix,2021-03-28,15:00:00,830,max_verstappen,1997-09-30,Dutch,18.0,2,0,9,28.0,\N,27674,28.0,2,0,red_bull,Austrian,33,1,2,2,2,18.0,56,+0.745,5524642,41,2,1:33.228,208.984,33,1,1:30.499,1:30.318,1:28.997,56.0,98654.321429,2.0,24.307500,19.0,3.000000,10.000000,1,0.500000,0,0.500000,10.000000,0,0,5.737500
8,1053,2021,2,21,Emilia Romagna Grand Prix,2021-04-18,13:00:00,830,max_verstappen,1997-09-30,Dutch,43.0,2,1,9,25.0,\N,27704,53.0,2,1,red_bull,Austrian,33,3,1,1,1,25.0,63,2:02:34.598,7354598,60,2,1:17.524,227.960,33,3,1:15.109,1:14.884,1:14.498,63.0,116739.650794,2.0,29.809000,NaN,NaN,NaN,1,NaN,1,NaN,NaN,1,0,5.691358
88,1054,2021,3,75,Portuguese Grand Prix,2021-05-02,14:00:00,830,max_verstappen,1997-09-30,Dutch,61.0,2,1,9,30.0,\N,27714,83.0,2,1,red_bull,Austrian,33,3,2,2,2,18.0,66,+29.148,5700569,62,3,1:20.695,207.581,33,3,1:19.485,1:18.650,1:18.746,66.0,86372.257576,4.0,24.471000,NaN,NaN,6.000000,1,0.750000,0,0.250000,7.666667,2,0,5.634146
108,1055,2021,4,4,Spanish Grand Prix,2021-05-09,13:00:00,830,max_verstappen,1997-09-30,Dutch,80.0,2,1,9,29.0,\N,27724,112.0,2,1,red_bull,Austrian,33,2,2,2,2,19.0,66,+15.841,5603521,62,1,1:18.149,215.357,33,2,1:18.090,1:16.922,1:16.777,66.0,84901.833333,2.0,22.578500,NaN,NaN,5.200000,1,0.800000,0,0.200000,2.000000,3,0,5.590361
128,1056,2021,5,6,Monaco Grand Prix,2021-05-23,13:00:00,830,max_verstappen,1997-09-30,Dutch,105.0,1,2,9,37.0,\N,27734,149.0,1,2,red_bull,Austrian,33,2,1,1,1,25.0,78,1:38:56.820,5936820,58,6,1:14.649,160.929,33,2,1:11.124,1:10.650,1:10.576,78.0,76113.076923,1.0,23.746000,NaN,NaN,4.666667,1,0.833333,1,0.166667,2.000000,4,0,5.547619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,1140,2024,20,32,Mexico City Grand Prix,2024-10-27,20:00:00,830,max_verstappen,1997-09-30,Dutch,362.0,1,7,9,8.0,\N,28933,512.0,3,7,red_bull,Austrian,1,2,6,6,6,8.0,71,+59.558,6115358,53,13,1:20.872,191.591,1,2,1:16.803,1:16.514,1:16.171,71.0,86131.802817,1.0,43.730000,1.0,1.666667,2.137500,0,0.850000,0,0.637500,3.333333,80,3,3.937500
1724,1141,2024,21,18,São Paulo Grand Prix,2024-11-03,17:00:00,830,max_verstappen,1997-09-30,Dutch,393.0,1,8,9,32.0,\N,28943,544.0,3,8,red_bull,Austrian,1,17,1,1,1,26.0,69,2:06:54.430,7614430,67,1,1:20.472,192.767,1,12,1:28.522,1:27.771,\N,69.0,110354.057971,1.0,NaN,3.0,3.666667,2.185185,1,0.839506,1,0.629630,3.666667,81,3,3.950311
1745,1142,2024,22,80,Las Vegas Grand Prix,2024-11-23,06:00:00,830,max_verstappen,1997-09-30,Dutch,403.0,1,8,9,11.0,\N,28953,555.0,3,8,red_bull,Austrian,1,5,5,5,5,10.0,50,+16.582,4942551,43,6,1:36.248,231.938,1,5,1:33.299,1:33.085,1:32.797,50.0,98851.020000,2.0,21.475500,1.0,2.000000,2.170732,0,0.841463,0,0.634146,3.333333,82,1,3.932099
1765,1143,2024,23,78,Qatar Grand Prix,2024-12-01,17:00:00,830,max_verstappen,1997-09-30,Dutch,429.0,1,9,9,26.0,\N,28963,581.0,3,9,red_bull,Austrian,1,2,1,1,1,25.0,57,1:31:05.323,5465323,55,2,1:22.905,235.310,1,1,1:21.579,1:20.687,1:20.520,57.0,95882.859649,3.0,

In [17]:
gp_to_nationality = {
    'Emilia Romagna Grand Prix': 'Italian',
    'Bahrain Grand Prix': 'Bahraini',
    'Qatar Grand Prix': 'Qatari',
    'Portuguese Grand Prix': 'Portuguese',
    'Spanish Grand Prix': 'Spanish',
    'Monaco Grand Prix': 'Monegasque',
    'Azerbaijan Grand Prix': 'Azerbaijani',
    'Styrian Grand Prix': 'Austrian',
    'French Grand Prix': 'French',
    'Austrian Grand Prix': 'Austrian',
    'British Grand Prix': 'British',
    'Hungarian Grand Prix': 'Hungarian',
    'Belgian Grand Prix': 'Belgian',
    'Dutch Grand Prix': 'Dutch',
    'Italian Grand Prix': 'Italian',
    'Russian Grand Prix': 'Russian',
    'Turkish Grand Prix': 'Turkish',
    'United States Grand Prix': 'American',
    'Mexico City Grand Prix': 'Mexican',
    'São Paulo Grand Prix': 'Brazilian',
    'Saudi Arabian Grand Prix': 'Saudi',
    'Abu Dhabi Grand Prix': 'Emirati',
    'Australian Grand Prix': 'Australian',
    'Miami Grand Prix': 'American',
    'Canadian Grand Prix': 'Canadian',
    'Singapore Grand Prix': 'Singaporean',
    'Japanese Grand Prix': 'Japanese',
    'Las Vegas Grand Prix': 'American',
    'Chinese Grand Prix': 'Chinese'
}
required_races['circuit_country_nationality'] = required_races['race_name'].map(gp_to_nationality)
required_races['is_home_gp'] = (
    required_races['circuit_country_nationality'] ==required_races['driver_nationality'].str.strip()
).astype(int)

In [18]:
required_races = required_races[required_races["year"].isin([2022, 2023, 2024])]
required_races.reset_index(drop=True, inplace=True)


In [29]:
required_races["race_name"].unique()

array(['Bahrain Grand Prix', 'Saudi Arabian Grand Prix',
       'Australian Grand Prix', 'Emilia Romagna Grand Prix',
       'Miami Grand Prix', 'Spanish Grand Prix', 'Monaco Grand Prix',
       'Azerbaijan Grand Prix', 'Canadian Grand Prix',
       'British Grand Prix', 'Austrian Grand Prix', 'French Grand Prix',
       'Hungarian Grand Prix', 'Dutch Grand Prix', 'Italian Grand Prix',
       'Singapore Grand Prix', 'Japanese Grand Prix',
       'United States Grand Prix', 'Mexico City Grand Prix',
       'São Paulo Grand Prix', 'Abu Dhabi Grand Prix',
       'Belgian Grand Prix', 'Las Vegas Grand Prix', 'Chinese Grand Prix',
       'Qatar Grand Prix'], dtype=object)

In [30]:
import pandas as pd
import requests
from datetime import datetime, timedelta
from time import sleep

race_name_to_coords = {
    'Bahrain Grand Prix': (26.0325, 50.5106),
    'Saudi Arabian Grand Prix': (21.6319, 39.1044),
    'Australian Grand Prix': (-37.8497, 144.9680),
    'Emilia Romagna Grand Prix': (44.3439, 11.7167),  # Imola, Italy
    'Miami Grand Prix': (25.9581, -80.2389),
    'Spanish Grand Prix': (41.5700, 2.2611),
    'Monaco Grand Prix': (43.7347, 7.4206),
    'Azerbaijan Grand Prix': (40.3725, 49.8533),
    'Canadian Grand Prix': (45.5000, -73.5228),
    'British Grand Prix': (52.0786, -1.0169),
    'Austrian Grand Prix': (47.2197, 14.7647),  # Red Bull Ring, Spielberg
    'French Grand Prix': (43.2508, 5.7917),  # Circuit Paul Ricard
    'Hungarian Grand Prix': (47.5789, 19.2486),
    'Dutch Grand Prix': (52.3889, 4.5400),
    'Italian Grand Prix': (45.6156, 9.2811),
    'Singapore Grand Prix': (1.2914, 103.8645),
    'Japanese Grand Prix': (34.8431, 136.5419),
    'United States Grand Prix': (30.1328, -97.6411),  # Circuit of the Americas
    'Mexico City Grand Prix': (19.4042, -99.0907),  # Autódromo Hermanos Rodríguez
    'São Paulo Grand Prix': (-23.7036, -46.6997),
    'Abu Dhabi Grand Prix': (24.4672, 54.6031),
    'Belgian Grand Prix': (50.4372, 5.9714),  # Spa-Francorchamps
    'Las Vegas Grand Prix': (36.1161, -115.1765),
    'Chinese Grand Prix': (31.3389, 121.2207),  # Shanghai International Circuit
    'Qatar Grand Prix': (25.49, 51.4542),
}


# 🌡️ Function to calculate average weather over a 90-minute race window
def average_weather(hours_list, start_time_str):
    try:
        start = datetime.strptime(start_time_str, "%H:%M:%S")
        end = start + timedelta(minutes=90)
        selected_hours = [
            h for h in hours_list
            if start <= datetime.strptime(h['datetime'], "%H:%M:%S") < end
        ]
        if not selected_hours:
            return None

        return {
            'avg_temp': sum(h['temp'] for h in selected_hours) / len(selected_hours),
            'avg_windspeed': sum(h['windspeed'] for h in selected_hours) / len(selected_hours),
            'avg_precip': sum(h['precip'] for h in selected_hours) / len(selected_hours),
            'avg_humidity': sum(h['humidity'] for h in selected_hours) / len(selected_hours),
        }
    except Exception as e:
        print(f"Error in averaging: {e}")
        return None

# 🔁 Main enrichment function
def enrich_with_weather(df, api_key):
    df = df.copy()

    # Add new columns for weather metrics
    df['avg_temp'] = None
    df['avg_windspeed'] = None
    df['avg_precip'] = None
    df['avg_humidity'] = None

    for idx, row in df.iterrows():
        race_name = row['race_name']
        race_date = row['race_date']
        race_time = row['race_time']

        coords = race_name_to_coords.get(race_name)
        if not coords:
            print(f"Unknown location for: {race_name}")
            continue

        lat, lon = coords
        location = f"{lat},{lon}"
        date_str = pd.to_datetime(race_date).strftime('%Y-%m-%d')

        # Request from Visual Crossing API
        try:
            url = (
                f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
                f"{location}/{date_str}?unitGroup=metric&include=hours&key={api_key}"
            )

            response = requests.get(url)
            if response.status_code != 200:
                print(f"API error for {race_name} on {date_str}: {response.status_code}, {response.text}")
                continue

            data = response.json()

            hours_data = data['days'][0]['hours']
            weather = average_weather(hours_data, race_time)

            if weather:
                df.at[idx, 'avg_temp'] = weather['avg_temp']
                df.at[idx, 'avg_windspeed'] = weather['avg_windspeed']
                df.at[idx, 'avg_precip'] = weather['avg_precip']
                df.at[idx, 'avg_humidity'] = weather['avg_humidity']
            else:
                print(f"No weather data for {race_name} on {date_str} at {race_time}")

            sleep(1)  # ⏳ Respect API rate limits

        except Exception as e:
            print(f"Failed to fetch weather for {race_name} on {race_date}: {e}")
            continue

    return df


In [32]:
required_races.sort_values(by=['race_date', 'race_time'], inplace=True)


In [42]:
twenty_22_Races = required_races[required_races["year"]==2022]
twenty_23_Races = required_races[required_races["year"]==2023]
twenty_24_Races = required_races[required_races["year"]==2024]

<class 'pandas.core.frame.DataFrame'>
Index: 386 entries, 21 to 1159
Data columns (total 58 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   raceId                                       386 non-null    int64  
 1   year                                         386 non-null    int64  
 2   round                                        386 non-null    int64  
 3   circuitId                                    386 non-null    int64  
 4   race_name                                    386 non-null    object 
 5   race_date                                    386 non-null    object 
 6   race_time                                    386 non-null    object 
 7   driverId                                     386 non-null    int64  
 8   driverRef                                    386 non-null    object 
 9   driver_dob                                   386 non-null    object 
 10  drive

In [37]:
import pandas as pd
import requests
from datetime import datetime, timedelta
from time import sleep

# 🎯 Circuit coordinates
race_name_to_coords = {
    'Bahrain Grand Prix': (26.0325, 50.5106),
    'Saudi Arabian Grand Prix': (21.6319, 39.1044),
    'Australian Grand Prix': (-37.8497, 144.9680),
    'Emilia Romagna Grand Prix': (44.3439, 11.7167),
    'Miami Grand Prix': (25.9581, -80.2389),
    'Spanish Grand Prix': (41.5700, 2.2611),
    'Monaco Grand Prix': (43.7347, 7.4206),
    'Azerbaijan Grand Prix': (40.3725, 49.8533),
    'Canadian Grand Prix': (45.5000, -73.5228),
    'British Grand Prix': (52.0786, -1.0169),
    'Austrian Grand Prix': (47.2197, 14.7647),
    'French Grand Prix': (43.2508, 5.7917),
    'Hungarian Grand Prix': (47.5789, 19.2486),
    'Dutch Grand Prix': (52.3889, 4.5400),
    'Italian Grand Prix': (45.6156, 9.2811),
    'Singapore Grand Prix': (1.2914, 103.8645),
    'Japanese Grand Prix': (34.8431, 136.5419),
    'United States Grand Prix': (30.1328, -97.6411),
    'Mexico City Grand Prix': (19.4042, -99.0907),
    'São Paulo Grand Prix': (-23.7036, -46.6997),
    'Abu Dhabi Grand Prix': (24.4672, 54.6031),
    'Belgian Grand Prix': (50.4372, 5.9714),
    'Las Vegas Grand Prix': (36.1161, -115.1765),
    'Chinese Grand Prix': (31.3389, 121.2207),
    'Qatar Grand Prix': (25.49, 51.4542),
}

# 🌡️ 90-minute weather average
def average_weather(hours_data, start_time_str):
    try:
        start = datetime.strptime(start_time_str, "%H:%M:%S")
        end = start + timedelta(minutes=90)
        selected = [h for h in hours_data if start <= datetime.strptime(h['time'], "%H:%M:%S") < end]

        if not selected:
            return None

        return {
            'avg_temp': sum(h['temp'] for h in selected) / len(selected),
            'avg_windspeed': sum(h['windspeed'] for h in selected) / len(selected),
            'avg_precip': sum(h['precip'] for h in selected) / len(selected),
            'avg_humidity': sum(h['humidity'] for h in selected) / len(selected),
        }

    except Exception as e:
        print(f"Error averaging weather: {e}")
        return None

# 🔁 Main enrichment function using Open-Meteo
def enrich_with_weather_openmeteo(df):
    df = df.copy()

    df['avg_temp'] = None
    df['avg_windspeed'] = None
    df['avg_precip'] = None
    df['avg_humidity'] = None

    for idx, row in df.iterrows():
        race_name = row['race_name']
        race_date = row['race_date']
        race_time = row['race_time']

        coords = race_name_to_coords.get(race_name)
        if not coords:
            print(f"Unknown location: {race_name}")
            continue

        lat, lon = coords
        date_str = pd.to_datetime(race_date).strftime('%Y-%m-%d')

        try:
            # Build Open-Meteo API request
            url = (
                f"https://archive-api.open-meteo.com/v1/archive?"
                f"latitude={lat}&longitude={lon}"
                f"&start_date={date_str}&end_date={date_str}"
                f"&hourly=temperature_2m,windspeed_10m,precipitation,relative_humidity_2m"
                f"&timezone=auto"
            )

            response = requests.get(url)
            if response.status_code != 200:
                print(f"API error {race_name}: {response.status_code}")
                continue

            json_data = response.json()
            if 'hourly' not in json_data:
                print(f"No hourly data for {race_name} on {date_str}")
                continue

            # Extract hour-wise data
            times = json_data['hourly']['time']
            hours_data = []
            for i in range(len(times)):
                hour_dt = datetime.strptime(times[i], '%Y-%m-%dT%H:%M')
                hours_data.append({
                    'time': hour_dt.strftime('%H:%M:%S'),
                    'temp': json_data['hourly']['temperature_2m'][i],
                    'windspeed': json_data['hourly']['windspeed_10m'][i],
                    'precip': json_data['hourly']['precipitation'][i],
                    'humidity': json_data['hourly']['relative_humidity_2m'][i],
                })

            weather = average_weather(hours_data, race_time)

            if weather:
                df.at[idx, 'avg_temp'] = weather['avg_temp']
                df.at[idx, 'avg_windspeed'] = weather['avg_windspeed']
                df.at[idx, 'avg_precip'] = weather['avg_precip']
                df.at[idx, 'avg_humidity'] = weather['avg_humidity']
            else:
                print(f"No matching hour window for {race_name} on {date_str}")

            sleep(1)

        except Exception as e:
            print(f"Failed for {race_name} on {race_date}: {e}")
            continue

    return df


In [48]:
api_key = "6VQN3VFDJ9RVGVPPNQB9VLZVZ"
sample_df = pd.DataFrame({
    'race_name': ['Bahrain Grand Prix', 'Hungarian Grand Prix'],
    'race_date': ['2023-03-05', '2024-07-21'],
    'race_time': ['15:00:00', '15:00:00']
})

enriched_df_3 = enrich_with_weather_openmeteo(twenty_24_Races)

In [49]:
enriched_df_3.to_csv(f"{base_path}2024_weather.csv")

In [51]:
required_races[['avg_temp', 'avg_windspeed', 'avg_precip', 'avg_humidity']] = np.nan


raceId  year  round  circuitId             race_name  \
race_date  race_time                                                         
2022-03-20 15:00:00     1074  2022      1          3    Bahrain Grand Prix   
           15:00:00     1074  2022      1          3    Bahrain Grand Prix   
           15:00:00     1074  2022      1          3    Bahrain Grand Prix   
           15:00:00     1074  2022      1          3    Bahrain Grand Prix   
           15:00:00     1074  2022      1          3    Bahrain Grand Prix   
...                      ...   ...    ...        ...                   ...   
2024-12-08 13:00:00     1144  2024     24         24  Abu Dhabi Grand Prix   
           13:00:00     1144  2024     24         24  Abu Dhabi Grand Prix   
           13:00:00     1144  2024     24         24  Abu Dhabi Grand Prix   
           13:00:00     1144  2024     24         24  Abu Dhabi Grand Prix   
           13:00:00     1144  2024     24         24  Abu Dhabi Grand Prix   

                      driverId   driverRef  driver_dob driver_nationality  \
race_date  race_time                                                        
2022-03-20 15:00:00          1    hamilton  1985-01-07            British   
           15:00:00          4      alonso  1981-07-29            Spanish   
           15:00:00        807  hulkenberg  1987-08-19             German   
           15:00:00        815       perez  1990-01-26            Mexican   
           15:00:00        817   ricciardo  1989-07-01         Australian   
...                        ...         ...         ...                ...   
2024-12-08 13:00:00        852     tsunoda  2000-05-11           Japanese   
           13:00:00        855        zhou  1999-05-30            Chinese   
           13:00:00        857     piastri  2001-04-06         Australian   
           13:00:00        859      lawson  2002-02-11      New Zealander   
           13:00:00        862      doohan  2003-01-20         Australian   

                      driverst_points  driverst_position  driverst_wins  \
race_date  race_time                                                      
2022-03-20 15:00:00              15.0                  3              0   
           15:00:00               2.0                  9              0   
           15:00:00               0.0                 17              0   
           15:00:00               0.0                 18              0   
           15:00:00               0.0                 14              0   
...                               ...                ...            ...   
2024-12-08 13:00:00              30.0                 12              0   
           13:00:00               4.0                 20              0   
           13:00:00             292.0                  4              2   
           13:00:00               4.0                 21              0   
           13:00:00               0.0                 24              0   

                      constructorId  csr_points csr_status  \
race_date  race_time                                         
2022-03-20 15:00:00             131        27.0         \N   
           15:00:00             214         8.0         \N   
           15:00:00             117         0.0         \N   
           15:00:00               9         0.0         \N   
           15:00:00               1         0.0         \N   
...                             ...         ...        ...   
2024-12-08 13:00:00             215         0.0         \N   
           13:00:00              15         0.0         \N   
           13:00:00               1        26.0         \N   
           13:00:00             215         0.0         \N   
           13:00:00             214         6.0         \N   

                      constructorStandingsId  css_points  css_position  \
race_date  race_time                                                     
2022-03-20 15:00:00                    28134        27.0             2   


In [69]:
hello = None
hello = required_races.copy()

hello.set_index(['race_date', 'driverId'], inplace=True)
enriched_df.set_index(['race_date', 'driverId'], inplace=True)
enriched_df_2.set_index(['race_date', 'driverId'], inplace=True)
enriched_df_3.set_index(['race_date', 'driverId'], inplace=True)
# Update from both enriched DataFrames
hello.update(enriched_df[['avg_temp', 'avg_windspeed', 'avg_precip', 'avg_humidity']])
hello.update(enriched_df_2[['avg_temp', 'avg_windspeed', 'avg_precip', 'avg_humidity']])
hello.update(enriched_df_3[['avg_temp', 'avg_windspeed', 'avg_precip', 'avg_humidity']])

# Reset index back to columns
hello.reset_index(inplace=True)


C:\Users\ibrah\AppData\Local\Temp\ipykernel_16728\1920864417.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[26.299999999999997 26.299999999999997 26.299999999999997 ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  hello.update(enriched_df[['avg_temp', 'avg_windspeed', 'avg_precip', 'avg_humidity']])
C:\Users\ibrah\AppData\Local\Temp\ipykernel_16728\1920864417.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[25.55 25.55 25.55 ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  hello.update(enriched_df[['avg_temp', 'avg_windspeed', 'avg_precip', 'avg_humidity']])
C:\Users\ibrah\AppData\Local\Temp\ipykernel_16728\1920864417.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future 

In [75]:
hello.to_csv(f"{base_path}final_dataset_for_cleaning.csv")